In [5]:
from sulcal_neuroanatomy import *

(262, 572, 0.458041958041958) (245, 572, 0.42832167832167833)
(225, 572, 0.39335664335664333) (112, 572, 0.1958041958041958)
(28, 572, 0.04895104895104895) (26, 572, 0.045454545454545456)
(413, 572, 0.722027972027972) (340, 572, 0.5944055944055944)
(450, 572, 0.7867132867132867) (490, 572, 0.8566433566433567)
(15, 572, 0.026223776223776224) (1, 572, 0.0017482517482517483)
(572, 572, 1.0) (572, 572, 1.0)
(241, 572, 0.42132867132867136) (257, 572, 0.4493006993006993)


In [6]:
all_subs_and_hems = making_tables_of_each_sub_and_hem(All_probabilities, subject_ids, hemispheres)

In [7]:
All_subject_and_hemisphere_sets = making_sets_of_relative_sulci_per_hemisphere_and_subject(all_subs_and_hems, sulci_names)

### Making sulci formulae

### Code for success of finding sulcus

### Precentral Sulcus

In [8]:
print(PrCS_prob_left, PrCS_prob_right)

(262, 572, 0.458041958041958) (245, 572, 0.42832167832167833)


In [9]:
PrCS_left_accuracy

({102715: 6,
  102816: 5,
  103111: 6,
  103212: 6,
  103414: 4,
  104416: 7,
  104820: 6,
  105115: 5,
  106319: 5,
  106521: 7,
  107018: 6,
  108525: 7,
  108828: 5,
  109123: 6,
  110411: 7,
  111312: 6,
  111413: 7,
  112112: 6,
  112819: 6,
  113619: 5,
  113821: 6,
  113922: 3,
  114419: 6,
  115017: 7,
  116524: 7,
  117324: 6,
  118124: 7,
  120717: 4,
  121315: 6,
  121416: 6,
  121618: 5,
  122620: 5,
  123117: 7,
  123420: 5,
  123925: 6,
  125525: 7,
  126931: 5,
  127630: 6,
  127832: 6,
  127933: 7,
  131217: 7,
  131419: 5,
  131621: 7,
  131823: 5,
  133625: 6,
  134021: 6,
  134324: 6,
  135124: 6,
  135225: 6,
  135528: 6,
  137431: 6,
  138534: 7,
  139435: 6,
  139839: 6,
  140420: 5,
  140925: 6,
  144226: 7,
  145632: 7,
  150524: 5,
  150625: 5,
  150726: 7,
  151223: 7,
  151425: 6,
  151930: 6,
  152225: 6,
  153025: 5,
  154835: 6,
  155231: 5,
  155635: 7,
  156031: 6,
  156233: 6,
  157336: 6,
  158843: 5,
  161630: 6,
  167440: 6,
  167743: 6,
  168139: 4,

In [10]:
PrCS_right_accuracy

({103111: 6,
  103212: 3,
  103515: 6,
  104012: 6,
  105115: 5,
  105216: 5,
  105620: 5,
  106319: 5,
  106521: 5,
  108121: 3,
  108323: 5,
  109123: 5,
  110411: 5,
  111413: 4,
  111716: 3,
  113215: 3,
  113619: 5,
  113922: 6,
  114419: 4,
  117122: 5,
  117324: 4,
  117930: 6,
  118124: 4,
  120717: 5,
  123420: 5,
  124220: 6,
  124826: 5,
  126931: 5,
  127832: 5,
  127933: 5,
  131722: 3,
  131823: 6,
  133019: 5,
  133625: 5,
  133827: 5,
  134324: 6,
  135124: 4,
  135528: 4,
  136833: 3,
  137431: 4,
  139839: 6,
  140420: 5,
  140925: 4,
  141826: 6,
  142424: 5,
  144226: 6,
  145632: 4,
  148133: 4,
  148941: 6,
  150524: 6,
  151425: 5,
  151627: 6,
  152225: 7,
  153025: 7,
  153429: 5,
  153833: 5,
  154431: 6,
  154734: 3,
  154936: 4,
  155231: 6,
  156637: 5,
  157336: 5,
  158540: 5,
  158843: 4,
  161327: 5,
  162733: 6,
  163129: 6,
  163331: 6,
  165032: 7,
  167440: 6,
  167743: 5,
  168139: 4,
  169444: 5,
  169747: 5,
  170934: 3,
  171330: 5,
  171633: 6,

### Superior Frontal Sulcus

In [11]:
print(SFS_prob_left, SFS_prob_right)

(225, 572, 0.39335664335664333) (112, 572, 0.1958041958041958)


In [12]:
SFS_left_accuracy

({103212: 1,
  103414: 1,
  103818: 1,
  104416: 1,
  104820: 1,
  105014: 1,
  105115: 1,
  106016: 1,
  106521: 1,
  107018: 1,
  107321: 1,
  108525: 1,
  108828: 1,
  110411: 1,
  111312: 1,
  113215: 1,
  114419: 1,
  114924: 2,
  118124: 1,
  121618: 1,
  122317: 1,
  123117: 1,
  124826: 1,
  125424: 2,
  127832: 1,
  127933: 1,
  131823: 1,
  133625: 1,
  134021: 1,
  135528: 1,
  136227: 1,
  136833: 1,
  145531: 1,
  145632: 1,
  146533: 1,
  148133: 1,
  151425: 1,
  151526: 1,
  151930: 1,
  152427: 1,
  152831: 1,
  153025: 1,
  153429: 1,
  153833: 1,
  154431: 1,
  154532: 1,
  154734: 1,
  154835: 1,
  155231: 1,
  155938: 1,
  157336: 1,
  158136: 1,
  161327: 1,
  162026: 1,
  162329: 1,
  163129: 1,
  165032: 1,
  167238: 1,
  167743: 1,
  169141: 1,
  171431: 1,
  172332: 1,
  173940: 1,
  175237: 1,
  176037: 1,
  176744: 1,
  177645: 1,
  177746: 1,
  180129: 1,
  180432: 1,
  180836: 1,
  181131: 1,
  181232: 1,
  182840: 1,
  185947: 1,
  186141: 1,
  188145: 1,

In [13]:
SFS_right_accuracy

({104012: 1,
  108121: 1,
  111312: 1,
  113619: 2,
  115017: 1,
  121618: 1,
  127630: 1,
  131419: 1,
  131621: 1,
  131924: 1,
  134021: 1,
  135225: 1,
  135528: 1,
  142424: 1,
  143325: 1,
  144226: 1,
  148133: 1,
  151526: 1,
  151930: 1,
  153126: 1,
  154835: 1,
  154936: 1,
  156233: 1,
  158843: 1,
  162026: 1,
  167743: 1,
  168947: 1,
  169343: 1,
  171431: 1,
  172534: 1,
  175237: 1,
  175439: 1,
  177746: 1,
  178849: 2,
  180129: 1,
  191942: 2,
  192843: 1,
  197449: 1,
  198249: 2,
  199150: 1,
  199251: 1,
  199655: 3,
  207123: 1,
  207426: 1,
  209834: 1,
  209935: 1,
  210617: 1,
  211114: 1,
  211922: 1,
  212116: 1,
  214019: 1,
  214726: 1,
  224022: 1,
  251833: 1,
  268850: 1,
  285446: 1,
  304020: 1,
  329440: 1,
  329844: 1,
  333330: 1,
  334635: 1,
  349244: 1,
  352132: 1,
  352738: 1,
  355542: 1,
  356948: 1,
  368551: 1,
  378756: 1,
  390645: 1,
  397760: 1,
  414229: 1,
  422632: 1,
  424939: 1,
  436845: 1,
  461743: 1,
  465852: 1,
  485757: 1,

### Calcarine Sulcus

In [14]:
print(Calcarine_prob_left, Calcarine_prob_right)

(28, 572, 0.04895104895104895) (26, 572, 0.045454545454545456)


In [15]:
Calcarine_left_accuracy

({103515: 2,
  106016: 2,
  108525: 1,
  114924: 1,
  124826: 3,
  131419: 2,
  134324: 1,
  152831: 1,
  154734: 2,
  158843: 2,
  182840: 2,
  336841: 2,
  355542: 3,
  356948: 3,
  371843: 1,
  441939: 2,
  485757: 2,
  541943: 3,
  573249: 2,
  592455: 2,
  679770: 1,
  705341: 3,
  742549: 1,
  774663: 1,
  870861: 3,
  930449: 2,
  958976: 1,
  994273: 2},
 9,
 0.32142857142857145)

In [16]:
Calcarine_right_accuracy

({113215: 1,
  114924: 1,
  115017: 1,
  118124: 2,
  131419: 2,
  150524: 1,
  152831: 1,
  158540: 2,
  167743: 1,
  171633: 1,
  173738: 1,
  181131: 3,
  185139: 3,
  197550: 1,
  255639: 2,
  339847: 2,
  355542: 2,
  536647: 1,
  566454: 1,
  679770: 1,
  729557: 2,
  748662: 1,
  774663: 1,
  898176: 2,
  899885: 1,
  930449: 2},
 15,
 0.5769230769230769)

### Postcentral Sulcus

In [17]:
print(Postcentral_prob_left, Postcentral_prob_right)

(413, 572, 0.722027972027972) (340, 572, 0.5944055944055944)


In [18]:
PoCS_left_accuracy

({102816: 6,
  103111: 5,
  103414: 6,
  103818: 6,
  104416: 6,
  104820: 5,
  105115: 5,
  105216: 5,
  105620: 5,
  106319: 6,
  106521: 4,
  107018: 5,
  107321: 7,
  108121: 4,
  108525: 6,
  109123: 4,
  109325: 6,
  110411: 5,
  111009: 7,
  111312: 6,
  111716: 6,
  112112: 4,
  112819: 6,
  113215: 6,
  113619: 6,
  113922: 5,
  114419: 6,
  114924: 2,
  115320: 5,
  116120: 6,
  116524: 5,
  117930: 5,
  118124: 6,
  118225: 5,
  120717: 6,
  121315: 6,
  121416: 6,
  122317: 6,
  122620: 5,
  122822: 5,
  123117: 4,
  123420: 5,
  123925: 6,
  124422: 6,
  124826: 6,
  125424: 5,
  125525: 6,
  126931: 6,
  127630: 5,
  127832: 6,
  127933: 6,
  130619: 5,
  130922: 5,
  131217: 5,
  131419: 6,
  131621: 5,
  131722: 6,
  131823: 5,
  133625: 5,
  133827: 5,
  133928: 4,
  134021: 5,
  135124: 6,
  136227: 4,
  136833: 4,
  137027: 6,
  137431: 6,
  138534: 6,
  139435: 5,
  139839: 5,
  140420: 5,
  140925: 6,
  141826: 6,
  143325: 5,
  144226: 7,
  145632: 6,
  146533: 6,

In [19]:
PoCS_right_accuracy

({103111: 4,
  103212: 4,
  103414: 5,
  103515: 4,
  103818: 3,
  105014: 5,
  105115: 6,
  105620: 5,
  106319: 6,
  106521: 5,
  107018: 4,
  107321: 5,
  108121: 5,
  108323: 2,
  108525: 5,
  109123: 4,
  109325: 6,
  111716: 6,
  112819: 5,
  113215: 5,
  113619: 6,
  113821: 5,
  114419: 4,
  116524: 4,
  117930: 3,
  118124: 4,
  120717: 6,
  121315: 4,
  121618: 4,
  123420: 6,
  123925: 5,
  124826: 5,
  125424: 6,
  125525: 4,
  126931: 5,
  127630: 5,
  127832: 8,
  127933: 6,
  130619: 4,
  131419: 6,
  131722: 6,
  131823: 4,
  132118: 4,
  133019: 6,
  133827: 6,
  133928: 2,
  135124: 6,
  135528: 6,
  135932: 6,
  136833: 5,
  137027: 5,
  137431: 5,
  138534: 4,
  139435: 4,
  140925: 7,
  141826: 6,
  142424: 4,
  143325: 4,
  144226: 3,
  145632: 3,
  146533: 5,
  148133: 5,
  148941: 4,
  150524: 6,
  151223: 4,
  151425: 6,
  152225: 3,
  153025: 6,
  153126: 6,
  154431: 6,
  154835: 4,
  154936: 3,
  155231: 7,
  155635: 6,
  155938: 5,
  156031: 4,
  156233: 6,

### Inferior Frontal Sulcus

In [20]:
print(IFS_prob_left, IFS_prob_right)

(450, 572, 0.7867132867132867) (490, 572, 0.8566433566433567)


In [21]:
IFS_left_accuracy

({102715: 11,
  103111: 10,
  103212: 8,
  103414: 9,
  103515: 8,
  103818: 9,
  104416: 9,
  104820: 8,
  105014: 8,
  105115: 8,
  105216: 10,
  105620: 10,
  106016: 9,
  106319: 8,
  106521: 10,
  107018: 9,
  107321: 8,
  107422: 8,
  108121: 10,
  108323: 9,
  108525: 9,
  108828: 10,
  109325: 9,
  110411: 9,
  111312: 9,
  111413: 8,
  111716: 9,
  112112: 10,
  112819: 7,
  113215: 9,
  113619: 8,
  113821: 9,
  114419: 9,
  114924: 9,
  115017: 9,
  115320: 10,
  116120: 8,
  116524: 9,
  117122: 8,
  117324: 8,
  117930: 10,
  118124: 8,
  120717: 8,
  121416: 8,
  121618: 9,
  122317: 8,
  122822: 8,
  123117: 9,
  123925: 7,
  124220: 10,
  124422: 8,
  124826: 8,
  125222: 9,
  125424: 9,
  125525: 9,
  126931: 8,
  127630: 8,
  127832: 7,
  130619: 9,
  131217: 7,
  131419: 9,
  131621: 9,
  131722: 9,
  131823: 7,
  131924: 9,
  132118: 10,
  133019: 8,
  133625: 9,
  133827: 9,
  133928: 9,
  134021: 8,
  134728: 9,
  135225: 7,
  135528: 9,
  135932: 9,
  136227: 9,


In [22]:
IFS_right_accuracy

({102715: 10,
  102816: 8,
  103111: 9,
  103212: 8,
  103414: 9,
  103515: 8,
  104012: 9,
  104416: 9,
  104820: 8,
  105014: 9,
  105115: 8,
  105216: 7,
  105620: 10,
  106319: 10,
  106521: 10,
  107321: 8,
  107422: 8,
  108121: 8,
  108323: 7,
  108525: 8,
  108828: 10,
  109123: 9,
  109325: 9,
  110411: 10,
  111009: 8,
  111312: 8,
  111413: 9,
  112112: 8,
  112819: 8,
  113215: 8,
  113619: 10,
  113821: 7,
  114419: 10,
  114924: 9,
  115017: 8,
  115320: 9,
  117930: 9,
  118124: 10,
  118225: 9,
  120717: 9,
  121315: 9,
  121416: 8,
  121618: 8,
  122317: 10,
  122822: 9,
  123117: 9,
  123420: 8,
  123925: 8,
  125222: 9,
  125525: 8,
  126931: 9,
  127630: 10,
  127832: 8,
  127933: 9,
  131419: 8,
  131621: 10,
  131823: 8,
  131924: 8,
  132118: 10,
  133019: 8,
  133625: 10,
  133827: 9,
  134021: 7,
  134324: 8,
  134728: 9,
  135124: 9,
  135225: 9,
  135528: 9,
  135932: 10,
  136227: 9,
  136833: 10,
  137027: 8,
  137431: 9,
  138534: 8,
  139435: 9,
  139839:

### Pericallosal Sulcus

In [23]:
print(Pericallosal_prob_left, Pericallosal_prob_right)

(15, 572, 0.026223776223776224) (1, 572, 0.0017482517482517483)


In [24]:
Pericallosal_left_accuracy

({108525: 3,
  135528: 3,
  145632: 4,
  154431: 4,
  207426: 3,
  217126: 3,
  236130: 5,
  257542: 5,
  355239: 3,
  568963: 5,
  585862: 2,
  671855: 2,
  705341: 5,
  782157: 5,
  994273: 2},
 0,
 0.0)

In [25]:
Pericallosal_right_accuracy

({715041: 9}, 0, 0.0)

### Lateral Occipit(o-Temporal) Sulcus

In [26]:
print(LOS_prob_left, LOS_prob_right)

(572, 572, 1.0) (572, 572, 1.0)


In [27]:
LOS_left_accuracy

({102715: 5,
  102816: 4,
  103111: 4,
  103212: 3,
  103414: 5,
  103515: 6,
  103818: 5,
  104012: 5,
  104416: 6,
  104820: 5,
  105014: 5,
  105115: 5,
  105216: 5,
  105620: 5,
  106016: 5,
  106319: 4,
  106521: 5,
  107018: 4,
  107321: 4,
  107422: 5,
  108121: 3,
  108323: 5,
  108525: 5,
  108828: 5,
  109123: 5,
  109325: 5,
  110411: 4,
  111009: 5,
  111312: 5,
  111413: 5,
  111716: 6,
  112112: 4,
  112819: 5,
  113215: 5,
  113619: 5,
  113821: 6,
  113922: 5,
  114419: 5,
  114924: 7,
  115017: 5,
  115320: 6,
  116120: 5,
  116524: 5,
  117122: 5,
  117324: 5,
  117930: 6,
  118124: 4,
  118225: 3,
  120717: 6,
  121315: 6,
  121416: 4,
  121618: 6,
  122317: 5,
  122620: 4,
  122822: 5,
  123117: 6,
  123420: 6,
  123925: 5,
  124220: 4,
  124422: 5,
  124826: 7,
  125222: 5,
  125424: 6,
  125525: 4,
  126931: 5,
  127630: 5,
  127832: 5,
  127933: 5,
  130619: 4,
  130922: 5,
  131217: 5,
  131419: 7,
  131621: 6,
  131722: 6,
  131823: 4,
  131924: 3,
  132118: 5,

In [28]:
LOS_right_accuracy

({102715: 5,
  102816: 5,
  103111: 5,
  103212: 5,
  103414: 5,
  103515: 5,
  103818: 5,
  104012: 6,
  104416: 5,
  104820: 5,
  105014: 4,
  105115: 4,
  105216: 4,
  105620: 5,
  106016: 5,
  106319: 5,
  106521: 5,
  107018: 6,
  107321: 5,
  107422: 5,
  108121: 6,
  108323: 5,
  108525: 6,
  108828: 6,
  109123: 6,
  109325: 6,
  110411: 6,
  111009: 5,
  111312: 8,
  111413: 5,
  111716: 5,
  112112: 6,
  112819: 4,
  113215: 5,
  113619: 4,
  113821: 4,
  113922: 4,
  114419: 4,
  114924: 7,
  115017: 5,
  115320: 6,
  116120: 6,
  116524: 6,
  117122: 5,
  117324: 5,
  117930: 5,
  118124: 5,
  118225: 5,
  120717: 5,
  121315: 6,
  121416: 3,
  121618: 3,
  122317: 2,
  122620: 4,
  122822: 4,
  123117: 5,
  123420: 5,
  123925: 6,
  124220: 3,
  124422: 5,
  124826: 6,
  125222: 4,
  125424: 5,
  125525: 6,
  126931: 5,
  127630: 4,
  127832: 5,
  127933: 5,
  130619: 5,
  130922: 5,
  131217: 5,
  131419: 5,
  131621: 5,
  131722: 3,
  131823: 5,
  131924: 7,
  132118: 5,

### Superior Occipital Sulcus

In [29]:
print(SOS_prob_left, SOS_prob_right)

(241, 572, 0.42132867132867136) (257, 572, 0.4493006993006993)


In [30]:
SOS_left_accuracy

({103111: 1,
  103212: 1,
  103414: 1,
  103818: 1,
  104012: 1,
  104820: 2,
  105216: 1,
  107018: 1,
  107321: 1,
  107422: 2,
  111009: 2,
  111312: 2,
  111716: 2,
  112112: 1,
  113215: 1,
  113619: 2,
  114419: 1,
  114924: 1,
  116524: 2,
  117122: 1,
  117324: 2,
  117930: 1,
  118124: 1,
  121618: 1,
  123117: 1,
  125424: 1,
  125525: 2,
  126931: 2,
  127933: 1,
  130922: 1,
  131621: 1,
  131722: 1,
  131823: 1,
  134324: 1,
  135528: 1,
  136833: 1,
  139435: 1,
  141826: 2,
  145531: 1,
  145632: 1,
  146533: 1,
  150625: 2,
  151728: 1,
  151930: 1,
  152427: 2,
  153227: 1,
  154431: 1,
  154734: 2,
  154835: 2,
  154936: 2,
  155231: 2,
  155635: 2,
  155938: 1,
  156233: 1,
  156637: 2,
  157336: 1,
  158843: 1,
  159239: 1,
  161327: 1,
  161630: 1,
  162026: 2,
  162329: 2,
  163129: 1,
  163331: 1,
  167238: 1,
  168341: 1,
  168947: 1,
  169141: 2,
  169444: 1,
  171330: 2,
  171431: 1,
  171633: 1,
  172029: 1,
  172938: 1,
  173334: 1,
  173940: 2,
  175035: 1,

In [31]:
SOS_right_accuracy

({102715: 1,
  103212: 1,
  103414: 1,
  104416: 1,
  105115: 1,
  105620: 2,
  106016: 1,
  106319: 1,
  106521: 1,
  107321: 1,
  107422: 1,
  108121: 2,
  109325: 1,
  111413: 1,
  112112: 1,
  113619: 1,
  113922: 1,
  114419: 1,
  114924: 1,
  115320: 1,
  116120: 1,
  116524: 1,
  117122: 1,
  117324: 1,
  117930: 1,
  118225: 1,
  121618: 1,
  122620: 1,
  123117: 1,
  123420: 1,
  123925: 1,
  125424: 1,
  125525: 1,
  126931: 2,
  127933: 1,
  131217: 1,
  131621: 1,
  131823: 1,
  132118: 1,
  133019: 1,
  133625: 1,
  133827: 1,
  133928: 1,
  134324: 1,
  135528: 1,
  136833: 1,
  137431: 1,
  138534: 2,
  140925: 1,
  141826: 1,
  143325: 1,
  148941: 1,
  150524: 1,
  150625: 1,
  151627: 1,
  152831: 1,
  153025: 1,
  153126: 1,
  153227: 1,
  153833: 1,
  154431: 1,
  154835: 1,
  154936: 1,
  155231: 1,
  156637: 1,
  157336: 1,
  158035: 1,
  158540: 1,
  158843: 1,
  159441: 1,
  162733: 1,
  163331: 2,
  165032: 1,
  168139: 1,
  169444: 1,
  171330: 2,
  171431: 1,